In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

# Custom neurons

In [42]:
class Neuron(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(Neuron, self).__init__(name=name)
    self.num_outputs = 1
    self.deltaU = tf.Variable(initial_value=tf.zeros([1,1]),trainable=False, name= self.name + "_adjustement")
    self.weights_init_val = weights_init_val
    self.b = tf.constant([[bias_init_val]]) ### TO DEFINE
    self.u = tf.Variable(initial_value=tf.zeros([1,1]),trainable=False, name= self.name + "_activation")

  def build(self, input_shape):
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,shape=[int(input_shape[-1]),self.num_outputs], name = self.name + "_weights")
    
  def call(self, inputs, temperature):
    net = tf.matmul(inputs, self.w)
    self.u.assign(1/(1 + tf.exp((net + self.b)/temperature)))
    return self.u.value()

In [43]:
neuron = Neuron("test_neuron",0.5, 0.0)
neuron

In [44]:
test = neuron(tf.ones([1,2]),temperature = 1)
test

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.26894143]], dtype=float32)>

In [48]:
neuron.w

<tf.Variable 'test_neuron/test_neuron_weights:0' shape=(2, 1) dtype=float32, numpy=
array([[0.5],
       [0.5]], dtype=float32)>

In [46]:
neuron.weights

[<tf.Variable 'test_neuron/test_neuron_weights:0' shape=(2, 1) dtype=float32, numpy=
 array([[0.5],
        [0.5]], dtype=float32)>,
 <tf.Variable 'test_neuron_adjustement:0' shape=(1, 1) dtype=float32, numpy=array([[0.]], dtype=float32)>,
 <tf.Variable 'test_neuron_activation:0' shape=(1, 1) dtype=float32, numpy=array([[0.26894143]], dtype=float32)>]

In [109]:
t1 =  tf.constant([[1]])
t2 =  tf.constant([[2]])

In [110]:
t1

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[1]], dtype=int32)>

In [111]:
t2 = tf.concat([t1,t2],1)
t2

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[1, 2]], dtype=int32)>

In [69]:
t2 = tf.concat([t2,t2],1)
t2

<tf.Tensor: shape=(1, 4), dtype=int32, numpy=array([[1, 2, 1, 2]], dtype=int32)>

In [55]:
t1 = tf.constant([[1],[2]])
t2 = tf.constant([[3],[4]])
t3 = tf.constant([[5,6]])

In [56]:
t1

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[1],
       [2]], dtype=int32)>

In [57]:
t3

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[5, 6]], dtype=int32)>

In [58]:
t1*t2

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[3],
       [8]], dtype=int32)>

---

# CUSTOM LAYERS

In [162]:
class InputLayer(tf.keras.layers.Layer):
  def __init__(self, name):
    super(InputLayer, self).__init__(name=name)
    
  def call(self, inputs):
    a1a1 = tf.reshape(inputs[0],shape=(1,1)) #1
    a1a2 = tf.reshape(inputs[1],shape=(1,1)) #2 
    a1a2a2 = tf.reshape(inputs[2],shape=(1,1)) #3
    a1a2a1 = tf.reshape(inputs[3],shape=(1,1)) #4
    a2a2 = tf.reshape(inputs[4],shape=(1,1)) #5
    a2a1 = tf.reshape(inputs[5],shape=(1,1)) #6
    a2a1a1 = tf.reshape(inputs[6],shape=(1,1)) #7
    a2a1a2 = tf.reshape(inputs[7],shape=(1,1)) #8

    to_hidden = []
    to_hidden.append((tf.concat([a1a2a2, a2a2], axis = 1))) #["TrRi"] [3,5]
    to_hidden.append((tf.concat([a1a2a1, a2a1], axis = 1))) #["RiTr"] [4,6] 
    to_hidden.append((tf.concat([a1a1, a2a1], axis = 1))) #["Plu"] [1,6] 
    to_hidden.append((tf.concat([a1a2, a1a2a2], axis = 1))) #["TTra"] [2,3] 
    to_hidden.append((tf.concat([a2a1, a2a1a1], axis = 1))) #["TTrb"] [6,7] 
    to_hidden.append((tf.concat([a1a2, a2a1], axis = 1))) #["Tr"] [2,6]
    to_hidden.append((tf.concat([a1a2a1, a2a1a2], axis = 1))) #["MeTr"] [4,8] 
    to_hidden.append((tf.concat([a1a2a1, a1a2], axis = 1))) #["MoTr"] [4,2] 
    to_hidden.append((tf.concat([a1a1, a1a2a1], axis = 1))) #["RiRi"] [1,4] 
    to_hidden.append((tf.concat([a1a1, a1a2], axis = 1))) #["MoFo"] [1,2] 
    to_hidden.append((tf.concat([a1a2a2, a1a2a1], axis = 1))) #["MMof"] [3,4] 
    to_hidden.append((tf.concat([a1a1, a1a2a2], axis = 1))) #["MoRi"] [1,3] 
    to_hidden.append((tf.concat([a1a2a2, a2a1a1], axis = 1))) #["MeRi"] [3,7] 
    to_hidden.append((tf.concat([a1a1, a2a2], axis = 1))) #["Ri"] [1,5] -

    output = tf.concat(to_hidden,axis = 0)
    return output

In [163]:
input_layer = InputLayer(name = "InputLayer")
input_layer

In [149]:
x = tf.constant([1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8])
x

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8], dtype=float32)>

In [164]:
res = input_layer(x)
res

<tf.Tensor: shape=(14, 2), dtype=float32, numpy=
array([[1.3, 1.5],
       [1.4, 1.6],
       [1.1, 1.6],
       [1.2, 1.3],
       [1.6, 1.7],
       [1.2, 1.6],
       [1.4, 1.8],
       [1.4, 1.2],
       [1.1, 1.4],
       [1.1, 1.2],
       [1.3, 1.4],
       [1.1, 1.3],
       [1.3, 1.7],
       [1.1, 1.5]], dtype=float32)>

In [165]:
res[0]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.3, 1.5], dtype=float32)>

In [170]:
tf.reshape(res[0], shape = (1,2))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1.3, 1.5]], dtype=float32)>

In [166]:
neuron = Neuron("test_neuron",0.5, 0.0)
neuron

In [171]:
test = neuron(tf.reshape(res[0], shape = (1,2)),temperature = 1)
test

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.1978161]], dtype=float32)>

In [41]:
class HiddenLayer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(HiddenLayer, self).__init__(name=name)
    dimensions = ['TrRi', 'RiTr', 'Plu', 'TTra', 'TTrb', 'Tr', 'MeTr', 'MoTr', 'RiRi', 'MoFo', 'MMof', 'MoRi', 'MeRi', 'Ri']
    self.neurons = []
    for string in dimensions:
      neuron = Neuron(name = "Hidden_"+string, weights_init_val = 0.5, bias_init_val=0.0)
      self.neurons.append(neuron)
    
  def call(self, inputs, temperature):

    activations = []
    dimensions = ['TrRi', 'RiTr', 'Plu', 'TTra', 'TTrb', 'Tr', 'MeTr', 'MoTr', 'RiRi', 'MoFo', 'MMof', 'MoRi', 'MeRi', 'Ri']
    for i in range(len(dimensions)):
      activations.append(self.neurons["Hidden_TrRi"](tf.reshape(inputs[0], shape = (1,2)),temperature = temperature))
    
    output = tf.concat(activations,axis = 0)
    return output

In [ ]:
class OutputLayer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(OutputLayer, self).__init__(name=name)
    dimensions = ['TrRi', 'RiTr', 'Plu', 'TTra', 'TTrb', 'Tr', 'MeTr', 'MoTr', 'RiRi', 'MoFo', 'MMof', 'MoRi', 'MeRi', 'Ri']
    self.neurons = {}
    for string in dimensions:
      neuron = Neuron(name = "Output_"+string, weights_init_val = 0.5, bias_init_val=0.0)
      self.neurons["Output_"+string](neuron)
    
    
  def call(self, inputs, temperature):
    
    return 

---

In [ ]:
strings = ['a1a1', 'a1a2', 'a1a2a2', 'a1a2a1', 'a2a2', 'a2a1', 'a2a1a1', 'a2a1a2']
dimensions = ['TrRi', 'RiTr', 'Plu', 'TTra', 'TTrb', 'Tr', 'MeTr', 'MoTr', 'RiRi', 'MoFo', 'MMof', 'MoRi', 'MeRi', 'Ri']

# CUSTOM MODEL

In [174]:
class MQSelfReflexiveNetwork(tf.keras.Model):
  def __init__(self):
    super(MQSelfReflexiveNetwork, self).__init__(name='MQSelfReflexiveNetwork')
    self.a1a1 = layers.Input(shape=())

    self.hidden = []
    self.output = []

    for string in strings:
      neuron = Neuron(name = "Hidden_"+string, weights_init_val = 0.5, bias_init_val=0.0)
      self.hidden.append(neuron)
    for string in strings:
      neuron = Neuron(name = "Output_"+string, weights_init_val = 0.5, bias_init_val=0.0)
      self.output.append(neuron)

    self.i = Neuron(name = "Hidden", weights_init_val = 0.5)
    self.j = Neuron(name = "Output", weights_init_val = 0.59)

    self.r = tf.constant([[1.0]]) # DEFINIRE
    self.temperature = tf.Variable(initial_value=tf.ones([1,1]),trainable=False)
    self.error = tf.Variable(initial_value=tf.zeros([1,1]),trainable=False)

  def call(self, input):
    ### FOWARD PASS
    hidden_layer_out = self.i(input, self.temperature)
    output_layer_out = self.j(hidden_layer_out, self.temperature)
    return tf.math.square(hidden_layer_out - output_layer_out)
  
  def train_step(self, input):
    ### CALL FOWARD PASS
    self.error.assign(self(input, training = True))
    
    ### BACK PROPAGATION
    # Compute temperature
    self.temperature.assign(1 - (1 / (1 + self.error) ))

    # Update outputs weights
    delta_Uj = self.j.u * (self.i.u - self.j.u) * (1 - self.j.u) + (self.j.deltaU * ((self.i.u - self.j.u)) )
    delta_Wij = self.r * delta_Uj * self.i.u
    self.j.w.assign_add(delta_Wij)
    ## Update outputs weights
    delta_Ui = self.i.u * (1 - self.i.u) * (delta_Uj * self.j.u)
    delta_Wki = self.r * delta_Ui * input
    self.i.w.assign_add(delta_Wki)

    return {
        "Error": tf.squeeze(self.error.value()),
        "Temperature": tf.squeeze(self.temperature.value()),
        "Hidden weight": tf.squeeze(self.i.w.value()),
        "Output weight": tf.squeeze(self.j.w.value())
        }

In [175]:
# Construct an instance of CustomModel
input = tf.ones([1,1])
model = MQSelfReflexiveNetwork()

In [176]:
model.call(input)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00448916]], dtype=float32)>

In [177]:
model.get_weights()

[array([[0.5]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[0.37754068]], dtype=float32),
 array([[0.59]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[0.]], dtype=float32),
 array([[0.44454187]], dtype=float32),
 array([[1.]], dtype=float32),
 array([[0.]], dtype=float32)]

In [178]:
model.compile()
model_history = model.fit(x = input, epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 91ms/step - Error: 0.0045 - Temperature: 0.0045 - Hidden weight: 0.4983 - Output weight: 0.5838
Epoch 2/200
1/1 [==============================] - 0s 4ms/step - Error: 0.2500 - Temperature: 0.2000 - Hidden weight: 0.4983 - Output weight: 0.5838
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - Error: 0.1354 - Temperature: 0.1193 - Hidden weight: 0.4954 - Output weight: 0.5768
Epoch 4/200
1/1 [==============================] - 0s 6ms/step - Error: 0.2170 - Temperature: 0.1783 - Hidden weight: 0.4946 - Output weight: 0.5750
Epoch 5/200
1/1 [==============================] - 0s 4ms/step - Error: 0.1552 - Temperature: 0.1344 - Hidden weight: 0.4921 - Output weight: 0.5693
Epoch 6/200
1/1 [==============================] - 0s 4ms/step - Error: 0.2012 - Temperature: 0.1675 - Hidden weight: 0.4908 - Output weight: 0.5665
Epoch 7/200
1/1 [==============================] - 0s 6ms/step - Error: 0.1653 - Temperature: 0.1419 - Hi

In [179]:
df_model_history = pd.DataFrame(model_history.history)
df_model_history

,Error,Temperature,Hidden weight,Output weight
0,0.004489,0.004469,0.498272,0.583754
1,0.250000,0.200000,0.498272,0.583754
2,0.135400,0.119253,0.495420,0.576806
3,0.217034,0.178330,0.494568,0.575009
4,0.155215,0.134361,0.492123,0.569271
...,...,...,...,...
195,0.000000,0.000000,-0.071070,-0.707804
196,0.000000,0.000000,-0.071070,-0.707804
197,0.000000,0.000000,-0.071070,-0.707804
198,0.000000,0.000000,-0.071070,-0.707804


In [182]:
fig = px.line(pd.DataFrame(model_history.history))
fig.update_layout(title="Monitored variables",
                   xaxis_title='Epoch',
                   yaxis_title='',
                   hovermode='x unified')
fig.show()

---